# Explore here

It's recommended to use this notebook for exploration purposes.

For example: 

1. You could import the CSV generated by python into your notebook and explore it.
2. You could connect to your database using `pandas.read_sql` from this notebook and explore it.

In [30]:
!pip install pandas
!pip install numpy
!pip install sklearn

  Using cached sklearn-0.0-py2.py3-none-any.whl
  Using cached scikit_learn-1.1.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (31.2 MB)
  Using cached scipy-1.8.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (41.6 MB)
  Using cached joblib-1.1.0-py2.py3-none-any.whl (306 kB)
  Using cached threadpoolctl-3.1.0-py3-none-any.whl (14 kB)


In [20]:
import pandas as pd
import numpy as np
import unicodedata
import re

In [5]:
url='https://raw.githubusercontent.com/4GeeksAcademy/naive-bayes-project-tutorial/main/playstore_reviews_dataset.csv'
df = pd.read_csv(url)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   package_name  891 non-null    object
 1   review        891 non-null    object
 2   polarity      891 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 21.0+ KB


In [8]:
df.sample(10)

,package_name,review,polarity
171,com.imangi.templerun2,awsome but i â¤â¤â¤â¤â¤â¤â¤â¤â¤â¤ t...,1
547,com.dropbox.android,"good app!!!i like dis app ,very useful i love...",1
386,com.google.android.talk,"bugs, bugs, and more bugs with each update. i...",0
235,com.supercell.clashofclans,this game đ.i donno y ppl give it 1 star ...,0
738,com.shirantech.kantipur,some notification problem although i've enabl...,0
709,com.opera.mini.native,love/hate has bug and security issues. i trie...,0
505,com.Slack,"need swipe between images nice app, but it wo...",0
721,com.opera.mini.native,best browser but keeps crashing i've no clue ...,0
110,com.linkedin.android,"still rubbish, still can't log in. fb inspire...",0
817,com.hamropatro,excellent!!!! awesome app very useful thanks...,1


Eliminamos los caracteres desconocidos

In [12]:
df['review']=df['review'].str.strip()
df['review']=df['review'].str.lower()

In [25]:
df['review']=df['review'].apply(normalize_str)
df['review']=df['review'].str.replace('!','')
df['review']=df['review'].str.replace(',','')
df['review']=df['review'].str.replace('&','')
df['review']=df['review'].str.normalize('NFKC')
df['review']=df['review'].str.replace(r'([a-zA-Z])\1{2,}',r'\1',regex=True)


In [21]:
def normalize_str(text_string):
    if text_string is not None:
        result=unicodedata.normalize('NFD',text_string).encode('ascii','ignore').decode()
    else:
        result=None 
    return result

In [26]:
df.sample(10)

,package_name,review,polarity
332,com.viber.voip,cant use not being able to call any contact pl...,0
502,com.Slack,great internal comms system for our business,1
140,com.king.candycrushsaga,was great i was going to give this 5 stars unt...,0
829,com.hamropatro,j khojyo thyehi paine ek matra apps,1
39,com.facebook.katana,app doesn't work after latest upgrade the face...,0
575,jabanaki.todo.todoly,i love this guy. he keep updating this simple ...,1
23,com.facebook.katana,i wish posts popped up in the order they were ...,0
200,com.supercell.clashofclans,force close it's been 1 or 2 months and y'all ...,0
438,com.whatsapp,no notification displayed on system notificati...,0
277,com.android.chrome,translation use it for browsing and visiting v...,0


In [27]:
df.iloc[675,]

package_name                                    com.hamrokeyboard
review          loved it rzrl app rrlrrl rrsrzrlrrl r r rsrzrl...
polarity                                                        1
Name: 675, dtype: object

In [28]:
X=df['review']
y=df['polarity']

In [31]:
from sklearn.model_selection import train_test_split

In [34]:
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=2007,stratify=y)

In [35]:
from sklearn.feature_extraction.text import CountVectorizer

In [36]:
vec=CountVectorizer(stop_words='english')
X_train=vec.fit_transform(X_train).toarray()
X_test=vec.transform(X_test).toarray()